In [1]:
from goyoai_ai_server.hub.model.chat import (
    OpenAIChatModelNamesEnum,
    get_chat_model,
)

chat_model = get_chat_model(OpenAIChatModelNamesEnum.GPT4O_MINI)

In [2]:
chat_model.invoke("hi")

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 8, 'total_tokens': 17, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': None, 'id': 'chatcmpl-BrdAJYosGrZGTH8R1vp2t9Xlkby88', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--3ee0a2d8-44e5-4384-859b-8f0306ecc8d2-0', usage_metadata={'input_tokens': 8, 'output_tokens': 9, 'total_tokens': 17, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [3]:
from typing import TypedDict, List
from langchain_core.messages import BaseMessage

class AgentState(TypedDict):
    messages: List[BaseMessage]

In [4]:
from goyoai_ai_server.services.intern.tools.web_search.perplexity.tool import PerplexityWebSearchTool

web_search_tool = PerplexityWebSearchTool()
tools = [web_search_tool]

In [5]:
from langchain_core.messages import ToolMessage
from langchain_core.runnables import RunnableLambda
from langchain_core.tools import tool

In [6]:
llm_with_tools = chat_model.bind_tools(tools)

def call_llm_node(state: AgentState) -> AgentState:
    response = llm_with_tools.invoke(state["messages"])
    print("🧠 LLM 응답:", response.content)
    print("🧩 Tool Calls:", getattr(response, "tool_calls", None))  # 👈 이게 있어야 tool 실행됨
    return {"messages": state["messages"] + [response]}

In [7]:
from langchain_core.runnables import RunnableLambda

def tool_executor(state: AgentState) -> AgentState:
    last_message = state['messages'][-1]
    if hasattr(last_message, "tool_calls"):
        new_messages = state["messages"]
        for call in last_message.tool_calls:
            tool_name = call["name"]
            args = call["args"]
            tool = next(t for t in tools if t.name == tool_name)
            result, _ = tool._run(**args)
            tool_msg = ToolMessage(content=result, tool_call_id=call["id"])
            new_messages.append(tool_msg)
        return {"messages" : new_messages}

    return state

In [8]:
from langgraph.graph import StateGraph

graph = StateGraph(state_schema=AgentState)

graph.add_node("call_llm", RunnableLambda(call_llm_node))
graph.add_node("call_tool", tool_executor)

graph.set_entry_point("call_llm")

def should_use_tool(state: AgentState) -> str:
    last_msg = state["messages"][-1]
    
    if hasattr(last_msg, "tool_calls") and last_msg.tool_calls:
        return "call_tool"
    return "end"


graph.add_conditional_edges("call_llm", should_use_tool)
graph.add_edge("call_tool", "call_llm")
graph.set_finish_point("call_llm")

runnable = graph.compile()

In [9]:
from langchain_core.messages import HumanMessage
from langchain_core.messages import SystemMessage

chat_history = [
    SystemMessage(
    content="너는 웹 검색 도구를 사용할 수 있어. 실시간 정보나 모르는 건 반드시 tool_call을 생성해서 도구를 사용한 후, 도구 응답(ToolMessage)을 참고해서 답변을 만들어야 해."
)
]

while True:
    user_input = input("👤 You: ")
    if user_input.lower() in {"exit", "quit"}:
        break

    chat_history.append(HumanMessage(content=user_input))
    result = runnable.invoke({"messages": chat_history})
    chat_history = result["messages"]

    print("🤖", chat_history[-1].content)
    
    

🧠 LLM 응답: 안녕하세요! 무엇을 도와드릴까요?
🧩 Tool Calls: []
🤖 안녕하세요! 무엇을 도와드릴까요?
🧠 LLM 응답: 
🧩 Tool Calls: [{'name': 'web_search', 'args': {'query': '오늘 날씨'}, 'id': 'call_rZkhVW0DrvIffANrKDLjza67', 'type': 'tool_call'}]
🧠 LLM 응답: 오늘의 날씨 정보는 제공되지 않지만, 기상청의 [날씨누리](https://www.weather.go.kr/w/weather/now.do) 웹사이트를 방문하면 현재 위치의 최신 날씨를 확인할 수 있습니다. 

날씨는 다음과 같은 기호로 설명됩니다:
- 맑음
- 구름조금
- 구름많음
- 흐림
- 소나기
- 비
- 눈
- 비 또는 눈

원하는 지역의 날씨를 확인하시려면 위 링크를 클릭하여 직접 확인하시기 바랍니다. 도움이 필요하면 말씀해 주세요!
🧩 Tool Calls: []
🤖 오늘의 날씨 정보는 제공되지 않지만, 기상청의 [날씨누리](https://www.weather.go.kr/w/weather/now.do) 웹사이트를 방문하면 현재 위치의 최신 날씨를 확인할 수 있습니다. 

날씨는 다음과 같은 기호로 설명됩니다:
- 맑음
- 구름조금
- 구름많음
- 흐림
- 소나기
- 비
- 눈
- 비 또는 눈

원하는 지역의 날씨를 확인하시려면 위 링크를 클릭하여 직접 확인하시기 바랍니다. 도움이 필요하면 말씀해 주세요!
🧠 LLM 응답: 더 궁금한 점이나 필요한 정보가 있으시면 언제든지 말씀해 주세요!
🧩 Tool Calls: []
🤖 더 궁금한 점이나 필요한 정보가 있으시면 언제든지 말씀해 주세요!
🧠 LLM 응답: 도움이 필요하시면 언제든지 말씀해 주세요! 어떤 정보를 찾고 계신가요?
🧩 Tool Calls: []
🤖 도움이 필요하시면 언제든지 말씀해 주세요! 어떤 정보를 찾고 계신가요?
🧠 LLM 응답: 어떤 정보가 필요하신지 말씀해 주시면 도와드리겠습니다. 질문이 있으시면 언제든지 말